In [113]:
import json
from sklearn.externals import joblib
import numpy as np
from azureml.core.model import Model
import pickle
import gensim
import pyodbc
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from azureml.core import Workspace
#ws = Workspace.create(name='saab_ml_workspace', subscription_id='047ae087-7d35-4c57-8fe9-7a442cc9cf16', resource_group='Speech_Analytics')

from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core import Workspace
svc_pr_password = "sWR5y]ocjTTMxT@7H13YXWFRcss=.nrN"

svc_pr = ServicePrincipalAuthentication(
    tenant_id="93f33571-550f-43cf-b09f-cd331338d086",
    service_principal_id="0a89e69e-7aae-4a80-8b76-7dc9f65c3d16",
    service_principal_password=svc_pr_password)


ws = Workspace(
    subscription_id="047ae087-7d35-4c57-8fe9-7a442cc9cf16",
    resource_group="Speech_Analytics",
    workspace_name="saab_ml_workspace",
    auth=svc_pr
    )

client_name = {'clientname' : 'SAAB'}
mysql = {'server': 'saab-server-resource.database.windows.net',
         'database': 'saab_dw_resource',
         'username': 'saabadmin',
         'password': 'p@$$w0rd',
         'driver':'{ODBC Driver 17 for SQL Server}',
         'PORT':'1433'}
client_details = {'Domain_ID':'101','UserName':'stripathy32','call_language1':'en'}

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    #model_path = Model.get_model_path('topic_model',5,ws)
    model = joblib.load("C:\\Sanjeeb\\Project\\SAAB\\Pickle Files\\Call Topics\\Topic_Modelling.pkl")
    #print(model)
    #return (model)
# Called when a request is received


In [118]:
def f(): 
    global s 
    print (s )
    s = "Look for Geeksforgeeks Python Section"
    print (s)  
  
#Global Scope 
s = "Python is great!" 
f() 
print (s)

Python is great!
Look for Geeksforgeeks Python Section
Look for Geeksforgeeks Python Section


In [115]:
python --version

NameError: name 'python' is not defined

In [114]:
model

NameError: name 'model' is not defined

In [110]:
def run(raw_data):
    
    
    df=pd.DataFrame(json.loads(raw_data))
    preprocessed_data=df[['Call_ID','Preprocessed Transcripts']]
    print(df[['Call_ID','Preprocessed Transcripts']])
    query="select calltopic_id,calltopics from [SAAB_ML_MASTER_CALLTOPICS_DM] where [DOMAIN_ID]="+client_details['Domain_ID'] +"order by calltopic_id"
    
    cnxn = pyodbc.connect('DRIVER='+mysql['driver']+';SERVER='+mysql['server']+';PORT='+mysql['PORT']+';DATABASE='+mysql['database']+';UID='+mysql['username']+';PWD='+ mysql['password'])

    cursor = cnxn.cursor()
    
    #Stored procedure to fetch the call id and transcripts for the respective call id.
    df_master_topics = pd.read_sql_query(query,cnxn)
    
    #Closing the connection
    cursor.close()
    cnxn.close()
       
    
    vectorizer = CountVectorizer(ngram_range = (3,3)) 
    X1=vectorizer.fit_transform(preprocessed_data['Preprocessed Transcripts'])
    trigrams=[]
    for text in preprocessed_data['Preprocessed Transcripts']:
        text=[text,]
        X1=(vectorizer.fit_transform(text))
        trigrams.append(vectorizer.get_feature_names()) 
    
    preprocessed_data['trigram_Transcript']=trigrams
    dictionary = gensim.corpora.Dictionary(preprocessed_data['trigram_Transcript'])
    bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_data['trigram_Transcript']]

    doc_lda = model[bow_corpus]
    results=[]
    for topic in doc_lda:
        results.append(topic)
    
    Number_Of_Topics=len(df_master_topics)
    num_topics=Number_Of_Topics
    #Getting the topics into a dataframe as column name and scores for each topic into cell per call
    cols=[]
    for i in range(0,num_topics):
        cols.append("Topic_"+str(i+1))
    df_Topics=pd.DataFrame(columns=cols)
    
    for index in range(len(results)):
        col_val=''
        for item in (results[index]):
            df_Topics.set_value(index, "Topic_"+str(item[0]+1), str(item[1]))
    
    df_Topics.columns=df_master_topics.calltopics
    preprocessed_data=pd.concat([preprocessed_data, df_Topics], axis=1)
    preprocessed_data.drop(['Preprocessed Transcripts'], axis=1, inplace=True)
    value_vars=df_master_topics.calltopics.to_list()
    df_result=pd.melt(preprocessed_data, id_vars =['Call_ID'], value_vars =value_vars ,
              var_name ='CallTopic', value_name ='CALLTOPIC_SCORE').dropna().sort_values('Call_ID')
    final_df=pd.merge(df_result, df_master_topics, how ='outer', left_on='CallTopic',right_on='calltopics')     
    
    final_df.sort_values(by=['Call_ID'],inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    
    final_df["Domain_ID"]=int(client_details['Domain_ID'])
    final_df['CREATED_BY']=client_details['UserName']
    final_df['RESULT_ID']=final_df.index+1
    final_df.drop(['CallTopic', 'calltopics'], axis=1, inplace=True)
    
    column_titles = ['RESULT_ID','Call_ID','Domain_ID','calltopic_id','CALLTOPIC_SCORE','CREATED_BY']
    final_df.reindex(columns=column_titles)
    
    
    out = final_df.to_json(orient='records')
    return out

In [111]:
return_df=run(out)

LdaModel(num_terms=4426, num_topics=4, decay=0.5, chunksize=2000)
LdaModel(num_terms=4426, num_topics=4, decay=0.5, chunksize=2000)
     Call_ID                           Preprocessed Transcripts
0   56482635  ['welcome', 'life', 'help', 'hello', 'child', ...
1   61876574  ['good', 'customer', 'care', 'help', 'today', ...
2   85550179  ['good', 'welcome', 'insurance', 'help', 'want...
3   98762981  ['good', 'customer', 'care', 'help', 'today', ...
4  114965010  ['graph', 'noon', 'life', 'customer', 'care', ...
Sanjeeb_1
Sanjeeb_2
Sanjeeb_3


C:\Users\stripathy32\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [119]:
pd.DataFrame(json.loads(return_df)) 

,CALLTOPIC_SCORE,CREATED_BY,Call_ID,Domain_ID,RESULT_ID,calltopic_id
0,0.98713654,stripathy32,56482635.0,101,1,1
1,0.98939246,stripathy32,61876574.0,101,2,3
2,0.9879721,stripathy32,85550179.0,101,3,4
3,0.9878804,stripathy32,98762981.0,101,4,3
4,0.9887876,stripathy32,114965010.0,101,5,3
5,None,stripathy32,NaN,101,6,2


In [3]:
import pandas as pd

In [4]:
df=pd.read_csv("Preprocessed_Transcripts_29012020161128.csv")

In [5]:
df.head()

,Call_ID,Raw Transcripts,Preprocessed Transcripts
0,56482635,Welcome to HDFC life my name is Kai how may I ...,"['welcome', 'life', 'help', 'hello', 'child', ..."
1,61876574,"Good afternoon, HDFC Life customer care. This ...","['good', 'customer', 'care', 'help', 'today', ..."
2,85550179,Good morning. Welcome to HDFC insurance? How c...,"['good', 'welcome', 'insurance', 'help', 'want..."
3,98762981,Good afternoon.\rHDFC life customer care this ...,"['good', 'customer', 'care', 'help', 'today', ..."
4,114965010,Graph noon AGC life customer care. This is Ale...,"['graph', 'noon', 'life', 'customer', 'care', ..."


In [104]:
df.dtypes

Call_ID                      int64
Raw Transcripts             object
Preprocessed Transcripts    object
dtype: object

In [15]:
df[['Call_ID','Preprocessed Transcripts']]

,Call_ID,Preprocessed Transcripts
0,56482635,"['welcome', 'life', 'help', 'hello', 'child', ..."
1,61876574,"['good', 'customer', 'care', 'help', 'today', ..."
2,85550179,"['good', 'welcome', 'insurance', 'help', 'want..."
3,98762981,"['good', 'customer', 'care', 'help', 'today', ..."
4,114965010,"['graph', 'noon', 'life', 'customer', 'care', ..."
5,136043333,"['hello', 'good', 'help', 'want', 'term_insura..."
6,162945426,"['good', 'welcome', 'life', 'insurance', 'help..."
7,163481267,"['good', 'life', 'customer', 'care', 'help', '..."
8,167214413,"['hello', 'ahdfc', 'life', 'help', 'want', 'kn..."
9,233293183,"['welcome', 'help', 'purchase', 'young', 'star..."


In [95]:
out = df[['Call_ID','Preprocessed Transcripts']].head(5).to_json(orient='records')#[1:-1].replace('},{', '} {')

In [96]:
out

'[{"Call_ID":56482635,"Preprocessed Transcripts":"[\'welcome\', \'life\', \'help\', \'hello\', \'child\', \'policy\', \'want\', \'know\', \'check\', \'phone\', \'value\', \'policy\', \'chill\', \'insurance\', \'purpose\', \'know\', \'policy_number\', \'angel\', \'policy_number\', \'second\', \'check\', \'value\', \'policy\', \'policy\', \'summary\', \'section\', \'account\', \'register\', \'line\', \'register\', \'facility\', \'registration_process\', \'icsi\', \'insurance\', \'registering\', \'procedure\', \'easy\', \'time\', \'registration\', \'process\', \'post\', \'continuous\', \'update\', \'fine\', \'register\', \'service\', \'type\', \'space\', \'policy_number\', \'alright\', \'unit\', \'link\', \'policy\', \'want\', \'inform\', \'assist\', \'maybe\', \'registration_process\', \'life\', \'pleasant\']"},{"Call_ID":61876574,"Preprocessed Transcripts":"[\'good\', \'customer\', \'care\', \'help\', \'today\', \'want\', \'unit_link\', \'plan\', \'help\', \'mobile_number\', \'angel\', 

AttributeError: 'str' object has no attribute 'to_list'

In [29]:
out

'[{"Call_ID":56482635,"Preprocessed Transcripts":"[\'welcome\', \'life\', \'help\', \'hello\', \'child\', \'policy\', \'want\', \'know\', \'check\', \'phone\', \'value\', \'policy\', \'chill\', \'insurance\', \'purpose\', \'know\', \'policy_number\', \'angel\', \'policy_number\', \'second\', \'check\', \'value\', \'policy\', \'policy\', \'summary\', \'section\', \'account\', \'register\', \'line\', \'register\', \'facility\', \'registration_process\', \'icsi\', \'insurance\', \'registering\', \'procedure\', \'easy\', \'time\', \'registration\', \'process\', \'post\', \'continuous\', \'update\', \'fine\', \'register\', \'service\', \'type\', \'space\', \'policy_number\', \'alright\', \'unit\', \'link\', \'policy\', \'want\', \'inform\', \'assist\', \'maybe\', \'registration_process\', \'life\', \'pleasant\']"},{"Call_ID":61876574,"Preprocessed Transcripts":"[\'good\', \'customer\', \'care\', \'help\', \'today\', \'want\', \'unit_link\', \'plan\', \'help\', \'mobile_number\', \'angel\', 

In [18]:
pd.read_json(out)

ValueError: Trailing data

In [19]:
import json

In [20]:
json.load(out)

AttributeError: 'str' object has no attribute 'read'

In [24]:
df[['Call_ID','Preprocessed Transcripts']].head(5).to_json('File.json')

In [22]:
import os

In [23]:
os.getcwd()

'C:\\Users\\stripathy32\\Saab_Dev_Codes'

In [25]:
dataframe_1=pd.read_json("File.json")

In [26]:
dataframe_1.head()

,Call_ID,Preprocessed Transcripts
0,56482635,"['welcome', 'life', 'help', 'hello', 'child', ..."
1,61876574,"['good', 'customer', 'care', 'help', 'today', ..."
2,85550179,"['good', 'welcome', 'insurance', 'help', 'want..."
3,98762981,"['good', 'customer', 'care', 'help', 'today', ..."
4,114965010,"['graph', 'noon', 'life', 'customer', 'care', ..."


In [30]:
y = json.loads(out)

In [32]:
y

[{'Call_ID': 56482635,
  'Preprocessed Transcripts': "['welcome', 'life', 'help', 'hello', 'child', 'policy', 'want', 'know', 'check', 'phone', 'value', 'policy', 'chill', 'insurance', 'purpose', 'know', 'policy_number', 'angel', 'policy_number', 'second', 'check', 'value', 'policy', 'policy', 'summary', 'section', 'account', 'register', 'line', 'register', 'facility', 'registration_process', 'icsi', 'insurance', 'registering', 'procedure', 'easy', 'time', 'registration', 'process', 'post', 'continuous', 'update', 'fine', 'register', 'service', 'type', 'space', 'policy_number', 'alright', 'unit', 'link', 'policy', 'want', 'inform', 'assist', 'maybe', 'registration_process', 'life', 'pleasant']"},
 {'Call_ID': 61876574,
  'Preprocessed Transcripts': "['good', 'customer', 'care', 'help', 'today', 'want', 'unit_link', 'plan', 'help', 'mobile_number', 'angel', 'mobile_number', 'want', 'know', 'want', 'information', 'unit_link', 'plan', 'know', 'premium', 'benefit', 'agree', 'confirmation',

In [99]:
dataf=pd.DataFrame(json.loads(out))

In [100]:
print(dataf.head())

     Call_ID                           Preprocessed Transcripts
0   56482635  ['welcome', 'life', 'help', 'hello', 'child', ...
1   61876574  ['good', 'customer', 'care', 'help', 'today', ...
2   85550179  ['good', 'welcome', 'insurance', 'help', 'want...
3   98762981  ['good', 'customer', 'care', 'help', 'today', ...
4  114965010  ['graph', 'noon', 'life', 'customer', 'care', ...


In [102]:
dataf[['Call_ID','Preprocessed Transcripts']]

,Call_ID,Preprocessed Transcripts
0,56482635,"'welcome', 'life', 'help', 'hello', 'child', '..."
1,61876574,"'good', 'customer', 'care', 'help', 'today', '..."
2,85550179,"'good', 'welcome', 'insurance', 'help', 'want'..."
3,98762981,"'good', 'customer', 'care', 'help', 'today', '..."
4,114965010,"'graph', 'noon', 'life', 'customer', 'care', '..."


In [97]:
dataf['Preprocessed Transcripts'] = dataf['Preprocessed Transcripts'].str.get(0)

In [101]:
dataf['Preprocessed Transcripts']=dataf['Preprocessed Transcripts'].str.strip('[]')

In [103]:
dataf.dtypes

Call_ID                      int64
Preprocessed Transcripts    object
dtype: object